# Simple Model
The aim of this workbook it to take an extremely simplified version of the model (just debris) and then build the equations and solve for them to ensure that the model is working correctly before implementing.

In [1]:
import pickle
import sympy as sp
import pandas as pd
import scipy as sc
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [672]:
with open(r'D:\ucl\pyssem\scenario_properties_2.pkl', 'rb') as f:
    scenario_properties = pickle.load(f)

In [673]:
# Define each of the variables that are required
time = scenario_properties.scen_times

# INITIAL POPULATION
# xo is the initial population, this shows for each orbital shell, the initial population
x0 = scenario_properties.x0.Su_260kg.to_list()
x0.append(0)

# LAUNCH RATE
# full lambda is the lambda value for the full time period
full_lambda = scenario_properties.full_lambda

# DRAG COEFFICIENT
# full_drag = scenario_properties.full_drag[:, 5]
# full_drag_lambda = []

# # Swap out the symbols for the actual values
# x, y = sp.symbols('x y')
# for i in range(len(full_drag)):
#     # Generate the current symbols to replace based on the expression's position in the list
#     sns_1 = sp.Symbol(f'N_6kg{i+1}')
#     sns_2 = sp.Symbol(f'N_6kg{i+2}')
    
#     expression = full_drag[i].subs({sns_1: x, sns_2: y})
    
#     # Update the list with the modified expression
#     full_drag_lambda.append(expression) 

# full_drag_updated_lambda = [sp.lambdify((x, y), eq, 'numpy') for eq in full_drag_lambda]

#Symbols
symbols = scenario_properties.species_types
print(symbols)

# Initial population
print(x0)

# Lambda function
print(full_lambda[0])

# POST MISSION DISPOSAL
full_pmd = scenario_properties.full_Cdot_PMD
symbols = full_pmd.atoms(sp.Symbol)
print(full_pmd[:, 1])

full_coll = scenario_properties.equations
print(full_coll[:, 1])

drag = scenario_properties.full_drag
print(drag[:, 1])



[]
[29, 234, 2343, 243, 32, 19, 21, 15, 358, 0]
[<scipy.interpolate._interpolate.interp1d object at 0x000002448E3DE180>, <scipy.interpolate._interpolate.interp1d object at 0x000002448E3DEDB0>, <scipy.interpolate._interpolate.interp1d object at 0x000002448E3DEC70>, <scipy.interpolate._interpolate.interp1d object at 0x000002448E3DE130>, <scipy.interpolate._interpolate.interp1d object at 0x000002448E3DEC20>, <scipy.interpolate._interpolate.interp1d object at 0x000002448E3DEE00>, <scipy.interpolate._interpolate.interp1d object at 0x000002448E1EA590>, <scipy.interpolate._interpolate.interp1d object at 0x000002448E1EAA40>, <scipy.interpolate._interpolate.interp1d object at 0x000002448E1EADB0>, <scipy.interpolate._interpolate.interp1d object at 0x000002448E1EAD60>]
Matrix([[-0.125*Su_473kg_1], [-0.125*Su_473kg_2], [-0.125*Su_473kg_3], [-0.125*Su_473kg_4], [-0.125*Su_473kg_5], [-0.125*Su_473kg_6], [-0.125*Su_473kg_7], [-0.125*Su_473kg_8], [-0.125*Su_473kg_9], [-0.125*Su_473kg_10]])
Matrix([[-2

In [675]:
symbols = sp.symbols(['Su_473kg_1', 'Su_473kg_2', 'Su_473kg_3', 'Su_473kg_4', 'Su_473kg_5', 'Su_473kg_6', 'Su_473kg_7', 'Su_473kg_8', 'Su_473kg_9', 'Su_473kg_10', 'sns_1', 'sns_2', 'sns_3', 'sns_4', 'sns_5', 'sns_6', 'sns_7', 'sns_8', 'sns_9', 'sns_10'])
lambdified_pmd = sp.lambdify(symbols, full_pmd[:, 1], 'numpy')
lambdified_coll = sp.lambdify(symbols, full_coll[:, 1], 'numpy')

def ode_system(t, y):
    try:
        launch_rates = [rate(t) for rate in full_lambda]
    except TypeError as e:
        # none type
        launch_rates = 0

    # Assuming full_lambda is a list of interp1d objects as indicated, 
    # and they are time-dependent launch rates, we evaluate them at the current time
    launch_rates = [rate(t) for rate in full_lambda]
    
    # Prepare the state with launch rates (assuming launch rates directly influence the state)
    # If this assumption is wrong, adjust accordingly
    state_with_launch_rates = np.hstack([y, launch_rates])
    
    # Evaluate the lambdified functions
    dydt_pmd = lambdified_pmd(*state_with_launch_rates)
    dydt_coll = lambdified_coll(*state_with_launch_rates)
    # Assuming drag contributions are zeros as per your last matrix, no need to add
    
    # Combine contributions (this depends on how you want to combine the effects)
    dydt = dydt_pmd + dydt_coll
    
    return dydt

# Time span for the ODE solution
t_span = [0, 10]  # Assuming you want to solve from t=0 to t=10, adjust as needed
t_eval = np.linspace(t_span[0], t_span[1], 100)  # Evaluation points

# Solve the ODE
solution = solve_ivp(ode_system, t_span, x0, t_eval=t_eval, method='RK45')


Error: 'list' object is not callable
Checking each element in full_lambda to identify the non-callable object.
Item at index 0 is not callable: [<scipy.interpolate._interpolate.interp1d object at 0x000002448E3DE180>, <scipy.interpolate._interpolate.interp1d object at 0x000002448E3DEDB0>, <scipy.interpolate._interpolate.interp1d object at 0x000002448E3DEC70>, <scipy.interpolate._interpolate.interp1d object at 0x000002448E3DE130>, <scipy.interpolate._interpolate.interp1d object at 0x000002448E3DEC20>, <scipy.interpolate._interpolate.interp1d object at 0x000002448E3DEE00>, <scipy.interpolate._interpolate.interp1d object at 0x000002448E1EA590>, <scipy.interpolate._interpolate.interp1d object at 0x000002448E1EAA40>, <scipy.interpolate._interpolate.interp1d object at 0x000002448E1EADB0>, <scipy.interpolate._interpolate.interp1d object at 0x000002448E1EAD60>]
Item at index 1 is not callable: None
Item at index 2 is not callable: None
Item at index 3 is not callable: [<scipy.interpolate._inter

TypeError: 'list' object is not callable

In [626]:
def satellite_ode(t, N, rate_of_change, full_drag, full_pmd):
    # Assuming rate_of_change is a callable that represents your interpolated rate of change function
    # N is going to be each shells current count of the number of satellites
    out = []
    
    # First we need to calculate the rate of change for each shell
    for i in range(len(N)):  # Adjust loop to avoid out-of-bounds
        
        # Calculate launch rate (input)
        r = rate_of_change[i](t)

        # Number of satellites decaying into the shell below
        sats_curr = N[i]
        try:
            sats_upp = N[i+1] # Number of satellites coming from the shell above
        except IndexError:
            sats_upp = 0 # Top shell
        
        # Now evaluate each lambdified drag equation
        drag_value = full_drag[i](sats_curr, sats_upp)

        # Post mission disposal
        pmd_value = full_pmd[i](sats_curr)

        out.append(r + drag_value + pmd_value)

    return out


# # Solve the system of ODEs
sol = solve_ivp(satellite_ode, [scenario_properties.scen_times[0], scenario_properties.scen_times[-1]], x0, 
                args=(full_lambda, full_drag_updated_lambda, full_pmd_lambda), 
                t_eval=scenario_properties.scen_times, method='RK23')

# Plotting the results
plt.figure(figsize=(10, 6))

for i in range(sol.y.shape[0]):
    plt.plot(sol.t, sol.y[i], label=f'Shell {i+1}')

plt.xlabel('Time')
plt.ylabel('Number of Satellites')
plt.title('Satellite Population Dynamics')
plt.legend()
plt.show()


TypeError: Cannot convert expression to float

In [554]:
# Now we need to do it for all species and all orbital shells at once
# Define each of the variables that are required
time = scenario_properties.scen_times

species = scenario_properties.species
species_order = []
for i in species.values():
    for j in i:
        species_order.append(j.sym_name)

# INITIAL POPULATION
# xo is the initial population, this shows for each orbital shell, the initial population
x0 = scenario_properties.x0

# as x0 doesn't have all of the values we need, we need to add the missing ones
x0_list = []
for species in species_order:
    if species in x0:
        list_ = x0[species].to_list()
        list_.append(0)
        x0_list.append(list_)
    else:
        x0_list.append([0] * scenario_properties.n_shells)

# LAUNCH RATE
# full lambda is the lambda value for the full time period
full_lambda = scenario_properties.full_lambda

# # DRAG COEFFICIENT
# full_drag = scenario_properties.full_drag
# full_drag_lambda = []

# PMD COEFFICIENT
full_pmd = scenario_properties.full_Cdot_PMD
# Convert into a list of lambdified functions
full_pmd_lambda = []
for i in full_pmd:
    print(i)

-0.125*Su_260kg_1
-0.125*Su_473kg_1
-0.125*S_148kg_1
-0.125*S_750kg_1
-0.125*S_1250kg_1
-0.333333333333333*sns_1
0
0
0.333333333333333*sns_1
0.0125*S_148kg_1
0.04375*Su_260kg_1
0.04375*Su_473kg_1
0.0125*S_750kg_1
0.0125*S_1250kg_1
0
-0.125*Su_260kg_2
-0.125*Su_473kg_2
-0.125*S_148kg_2
-0.125*S_750kg_2
-0.125*S_1250kg_2
-0.333333333333333*sns_2
0
0
0.333333333333333*sns_2
0.0125*S_148kg_2
0.04375*Su_260kg_2
0.04375*Su_473kg_2
0.0125*S_750kg_2
0.0125*S_1250kg_2
0
-0.125*Su_260kg_3
-0.125*Su_473kg_3
-0.125*S_148kg_3
-0.125*S_750kg_3
-0.125*S_1250kg_3
-0.333333333333333*sns_3
0
0
0.333333333333333*sns_3
0.0125*S_148kg_3
0.04375*Su_260kg_3
0.04375*Su_473kg_3
0.0125*S_750kg_3
0.0125*S_1250kg_3
0
-0.125*Su_260kg_4
-0.125*Su_473kg_4
-0.125*S_148kg_4
-0.125*S_750kg_4
-0.125*S_1250kg_4
-0.333333333333333*sns_4
0
0
0.333333333333333*sns_4
0.0125*S_148kg_4
0.04375*Su_260kg_4
0.04375*Su_473kg_4
0.0125*S_750kg_4
0.0125*S_1250kg_4
0
-0.125*Su_260kg_5
-0.125*Su_473kg_5
-0.125*S_148kg_5
-0.125*S_750kg_

In [550]:
def satellite_ode_multi_species(t, N, rate_of_change, full_pmd):
    out = []
    
    # Calculate the number of shells (assuming each species has the same number of shells)
    n_shells = len(x0_list[0])

    print(n_shells)
    
    # Calculate for each species and each shell
    for species_index in range(len(species_order)):
        for shell_index in range(n_shells):  # Adjusted to n_shells
            print(shell_index)
            index = species_index * n_shells + shell_index
            
            print(index)
            # Calculate launch rate (input)
            r = rate_of_change[species_index][shell_index](t)

            print(r)
            
            sats_curr = N[index]
            sats_upp = N[index + 1] if shell_index < n_shells - 1 else 0  # Handling last shell correctly

            print(sats_curr, sats_upp)
            
            # Drag and PMD values
            #drag_value = full_drag[species_index](sats_curr, sats_upp) if shell_index < n_shells - 1 else 0  # Adjusted logic
            
            # Now evaluate each lambdified drag equation
            # print(species_index, sats_curr, sats_upp)
            # drag_value = full_drag[species_index](sats_curr, sats_upp)

            pmd_value = full_pmd[species_index](sats_curr)
            
            # Append the calculated value
            out.append(r + pmd_value)
    
    return np.array(out)  # Ensure the return type is a NumPy array for compatibility with solve_ivp


N_initial = [item for sublist in x0_list for item in sublist]

print(N_initial)

sol = solve_ivp(satellite_ode_multi_species, [time[0], time[-1]], N_initial, args=(full_lambda, full_pmd_lambda), t_eval=time, method='RK23')

[2, 2, 11, 14, 30, 43, 44, 117, 161, 212, 265, 1705, 186, 31, 20, 6, 21, 2, 3, 6, 1, 0, 11, 7, 16, 4, 0, 1, 2, 5, 0, 8, 44, 286, 28, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 1, 3, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

TypeError: 'NoneType' object is not subscriptable

In [571]:
scenario_properties.equations[:, 5]
symbols = scenario_properties.equations[:, 5].free_symbols
print(symbols)


{S_750kg_20, N_260kg_27, N_750kg_40, N_148kg_30, N_1250kg_28, Su_473kg_15, Su_473kg_5, Su_260kg_9, N_148kg_10, N_1250kg_2, N_6kg_10, S_750kg_18, Su_260kg_32, N_260kg_38, N_473kg_30, N_6kg_1, N_0.00141372kg_18, Su_260kg_3, N_473kg_13, N_0.00141372kg_8, N_148kg_32, N_473kg_7, Su_473kg_27, S_1250kg_35, N_0.00141372kg_6, S_148kg_23, N_260kg_37, N_750kg_31, N_473kg_17, N_750kg_26, B_15, N_1250kg_26, N_0.567kg_4, N_0.567kg_14, N_1250kg_16, S_750kg_21, S_148kg_31, S_750kg_8, N_0.00141372kg_34, Su_473kg_28, N_260kg_2, N_750kg_22, N_750kg_8, B_33, N_6kg_40, Su_473kg_31, S_148kg_27, N_148kg_34, N_0.567kg_7, S_750kg_14, N_260kg_11, Su_473kg_17, Su_260kg_15, N_148kg_1, N_148kg_33, N_0.567kg_37, B_1, N_0.567kg_3, N_6kg_29, N_0.00141372kg_10, N_260kg_6, N_6kg_15, sns_8, N_0.567kg_15, Su_473kg_3, N_473kg_4, S_1250kg_21, B_17, N_750kg_19, sns_27, S_750kg_17, N_473kg_6, N_1250kg_14, S_148kg_17, N_473kg_21, N_0.567kg_11, N_1250kg_13, N_260kg_28, N_473kg_29, N_0.00141372kg_30, B_32, S_148kg_36, Su_260kg_

In [595]:
scenario_properties.equations[0,:]

symbols = scenario_properties.equations[:, 5].free_symbols
equations_func = [sp.lambdify(symbols, eq, 'numpy') for eq in scenario_properties.equations[0,:]]

D:\TEMP\ipykernel_9284\4283104317.py:4: SymPyDeprecationWarning: 

Passing the function arguments to lambdify() as a set is deprecated. This
leads to unpredictable results since sets are unordered. Instead, use a list
or tuple for the function arguments.

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-lambdify-arguments-set
for details.

This has been deprecated since SymPy version 1.6.3. It
will be removed in a future version of SymPy.

  equations_func = [sp.lambdify(symbols, eq, 'numpy') for eq in scenario_properties.equations[0,:]]


species  N_0.00141372kg  Su_260kg
alt_bin                          
0.0                   0        29
1.0                   0       234
2.0                   1      2343
3.0                   0       243
4.0                   6        32
5.0                   3        19
6.0                   2        21
7.0                   0        15
8.0                   0       358
[[<scipy.interpolate._interpolate.interp1d object at 0x00000244FFAE3720>, <scipy.interpolate._interpolate.interp1d object at 0x00000244FFAE39A0>, <scipy.interpolate._interpolate.interp1d object at 0x00000244FFAE3B80>, <scipy.interpolate._interpolate.interp1d object at 0x00000244FFAE3A40>, <scipy.interpolate._interpolate.interp1d object at 0x00000244FFAE3EF0>, <scipy.interpolate._interpolate.interp1d object at 0x0000024494B46F90>, <scipy.interpolate._interpolate.interp1d object at 0x0000024494B46D60>, <scipy.interpolate._interpolate.interp1d object at 0x0000024494B46B30>, <scipy.interpolate._interpolate.interp1d object 